In [20]:
#importing  headers
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [21]:
base_url = "https://houseprice.ie/search?county=Dublin&page="
details_base = "https://houseprice.ie"
properties = []

In [ ]:
def scrape_page(page):
    response = requests.get(url)
    print(f"✅ Page {page} loaded")

    soup = BeautifulSoup(response.content, 'html.parser')
    listings = soup.select('div.py-4.border-b.border-neutral-200')
    print(f"Found {len(listings)} listings on page {page}")

    page_properties = []
    for listing in listings:
        address_tag = listing.find('h3', class_='text-lg font-medium text-secondary-700 mb-1')
        if not address_tag:
            continue
        address = address_tag.get_text(strip=True)

        info_div = listing.find('div', class_='flex flex-wrap items-center gap-x-3 gap-y-1 text-sm text-neutral-600')
        if not info_div:
            continue
        info_texts = [x.get_text(strip=True) for x in info_div.find_all(['span', 'a']) if x.get_text(strip=True) not in ['•', '']]
        price = listing.find('div', class_="text-xl font-bold text-primary-600")
        price = price.get_text(strip=True) if price else None

        county = area = sale_date = house_type = sale_type = eircode = None
        ber_rating = bedrooms = bathrooms = floorarea = None

        for text in info_texts:
            if re.search(r'\d{1,2} \w+ \d{4}', text):
                sale_date = text
            elif re.match(r'Dublin', text):
                county = text
            elif re.search(r'^(Apartment|Bungalow|Duplex|Terrace|Semi-Detached|Detached|End of Terrace)', text):
                house_type = text
            elif re.search(r'^(New Build|Resale)', text):
                sale_type = text
            elif re.match(r'^[A-Z0-9]{3,7}$', text):
                eircode = text
            elif (
                not any(k in text for k in ['Apartment', 'Bungalow', 'Detached', 'Semi', 'Terrace', 'House', 'Duplex'])
                and not re.search(r'\d', text)
                and not area):
                area = text

        link_tag = listing.find('a', string=re.compile("View Details"))
        details_link = details_base + link_tag['href'] if link_tag else None
        if details_link:
            details_response = requests.get(details_link)
            details_soup = BeautifulSoup(details_response.content, 'html.parser')
            details = details_soup.select('div.flex.justify-between')

            for d in details:
                label = d.find('span', class_='font-medium text-neutral-600')
                value = d.find('span', class_='text-neutral-800')
                if not label or not value:
                    continue
                label_text = label.get_text(strip=True)
                value_text = value.get_text(strip=True)

                if "BER" in label_text:
                    ber_rating = value_text
                elif "Property Type" in label_text:
                    house_type = value_text
                elif "Bedrooms" in label_text:
                    bedrooms = value_text
                elif "Bathrooms" in label_text:
                    bathrooms = value_text
                elif "Size" in label_text:
                    floorarea = value_text

        page_properties.append({
            "address": address,
            "county": county,
            "area": area,
            "sale_date": sale_date,
            "house_type": house_type,
            "sale_type": sale_type,
            "eircode": eircode,
            "BER": ber_rating,
            "Bedrooms": bedrooms,
            "Bathrooms": bathrooms,
            "Floor Area": floorarea,
            "price": price
        })

    return page_properties


In [23]:
for page in range(300,665):  # test with first 5 pages
    props = scrape_page(page)
    properties.extend(props)
    time.sleep(2)  # wait 2 seconds between pages to avoid blocking
    print(f"Total properties collected: {len(properties)}")

✅ Page 300 loaded
Found 50 listings on page 300
Total properties collected: 50
✅ Page 301 loaded
Found 50 listings on page 301
Total properties collected: 100
✅ Page 302 loaded
Found 50 listings on page 302
Total properties collected: 150
✅ Page 303 loaded
Found 50 listings on page 303
Total properties collected: 200
✅ Page 304 loaded
Found 50 listings on page 304
Total properties collected: 250
✅ Page 305 loaded
Found 50 listings on page 305
Total properties collected: 300
✅ Page 306 loaded
Found 50 listings on page 306
Total properties collected: 350
✅ Page 307 loaded
Found 50 listings on page 307
Total properties collected: 400
✅ Page 308 loaded
Found 50 listings on page 308
Total properties collected: 450
✅ Page 309 loaded
Found 50 listings on page 309
Total properties collected: 500
✅ Page 310 loaded
Found 50 listings on page 310
Total properties collected: 550
✅ Page 311 loaded
Found 50 listings on page 311
Total properties collected: 600
✅ Page 312 loaded
Found 50 listings on pa

In [24]:
df = pd.DataFrame(properties)
df.to_csv('dublin_properties_20251.csv', index=False)
print(f"✅ Saved {len(df)} listings to dublin_properties_2025.csv")


✅ Saved 18250 listings to dublin_properties_2025.csv


In [27]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time
from tqdm import tqdm  # text-based progress bar

# --- Load your scraped dataset ---
df = pd.read_csv("dublin_properties_20251.csv")
print(f"Loaded {len(df)} listings")

# --- Initialize geocoder ---
geolocator = Nominatim(user_agent="dublin_transport_app")

# --- Get Dublin City Centre coordinates once ---
city_centre = geolocator.geocode("City Centre, Dublin, Ireland")
if not city_centre:
    raise ValueError("City Centre not found.")
city_coords = (city_centre.latitude, city_centre.longitude)
print("✅ City Centre Coordinates:", city_coords)

# --- Function to compute distance from city centre ---
def compute_distance_from_centre(area_name):
    try:
        location = geolocator.geocode(f"{area_name}, Dublin, Ireland", timeout=10)
        if not location:
            return None
        area_coords = (location.latitude, location.longitude)
        distance_km = geodesic(city_coords, area_coords).km
        return round(distance_km, 2)
    except Exception as e:
        print(f"⚠️ Distance calc failed for {area_name}: {e}")
        return None
# --- Function to find nearby Luas/DART stations ---
def find_nearby_transport(area_name):
    try:
        location = geolocator.geocode(f"{area_name}, Dublin, Ireland", timeout=10)
        if not location:
            return None
        lat, lon = location.latitude, location.longitude

        overpass_url = "http://overpass-api.de/api/interpreter"
        query = f"""
        [out:json];
        (
          node["railway"="station"](around:1000,{lat},{lon});
          node["railway"="tram_stop"](around:1000,{lat},{lon});
        );
        out;
        """
        response = requests.get(overpass_url, params={'data': query}, timeout=20).json()

        stations = []
        if 'elements' in response:
            for element in response['elements']:
                name = element['tags'].get('name', 'Unknown')
                if element['tags'].get('railway') == 'station':
                    st_type = "DART/Rail"

                elif element['tags'].get('railway') == 'tram_stop':
                    st_type = "Luas"
                else:
                    st_type = "Other"
                stations.append(f"{name} ({st_type})")
                print(f"{area_name} → {name} → {st_type} Station")

        return "; ".join(stations) if stations else None
    except Exception as e:
        print(f"⚠️ Transport lookup failed for {area_name}: {e}")
        return None

# --- Add new columns ---
df["Nearby_Transport"] = None
df["Distance_from_CityCentre_km"] = None

# --- Use only unique areas to reduce geocoding calls ---
unique_areas = df["area"].dropna().unique()
area_cache = {}

print(f"🌍 Found {len(unique_areas)} unique areas")

# --- Precompute for each area ---
for area in tqdm(unique_areas, desc="Geocoding areas"):
    transport = find_nearby_transport(area)
    distance = compute_distance_from_centre(area)
    area_cache[area] = {
        "Nearby_Transport": transport,
        "Distance_from_CityCentre_km": distance
    }
    time.sleep(1)  # respect API limits

# --- Map results back to full dataframe ---
df["Nearby_Transport"] = df["area"].map(lambda x: area_cache.get(x, {}).get("Nearby_Transport"))
df["Distance_from_CityCentre_km"] = df["area"].map(lambda x: area_cache.get(x, {}).get("Distance_from_CityCentre_km"))

# --- Save updated dataset ---
df.to_csv("dublin_properties_with_area_transport_distance1.csv", index=False)
print("✅ Saved enriched dataset → dublin_properties_with_area_transport_distance.csv")

# --- Preview sample ---
df.head()


Loaded 18250 listings
✅ City Centre Coordinates: (53.3970221, -6.2387185)
🌍 Found 228 unique areas


Geocoding areas:   0%|          | 0/228 [00:00<?, ?it/s]

Skerries → Skerries → DART/Rail Station


Geocoding areas:   1%|          | 2/228 [00:06<13:22,  3.55s/it]

⚠️ Transport lookup failed for Leopardstown: Expecting value: line 1 column 1 (char 0)


...

Geocoding areas:  99%|█████████▊| 225/228 [16:18<00:15,  5.05s/it]

Maynooth → Maynooth → DART/Rail Station


Geocoding areas: 100%|██████████| 228/228 [16:25<00:00,  4.32s/it]


✅ Saved enriched dataset → dublin_properties_with_area_transport_distance.csv


,address,county,area,sale_date,house_type,sale_type,eircode,BER,Bedrooms,Bathrooms,Floor Area,price,Nearby_Transport,Distance_from_CityCentre_km
0,"10 Moorings Cove, Balbriggan Street, Skerries",Dublin,Skerries,17 December 2024,Apartment,Resale,K34P650,C3,2.0,1.0,60 m²,"€315,000",Skerries (DART/Rail),21.23
1,"10 Churchfields Avenue, Mulhuddart, Dublin 15",Dublin,Mulhuddart,17 December 2024,NaN,New Build,NaN,NaN,NaN,NaN,NaN,"€399,974†",None,10.63
2,"12 Glencairn Drive, The Gallops, Leopardstown ...",Dublin,Leopardstown,17 December 2024,Semi-D,Resale,D18E6C8,C1,4.0,3.0,NaN,"€1,125,000",None,13.95
3,"13 The Drive, Carberry Meadows, Balmoston",Dublin,Balmoston,17 December 2024,NaN,New Build,NaN,NaN,NaN,NaN,NaN,"€530,000†",None,11.98
4,"13 The Lawns, Kilmartin Groe, Hollystown",Dublin,Hollystown,17 December 2024,NaN,New Build,NaN,NaN,NaN,NaN,NaN,"€495,000†",None,9.69


In [30]:
# Data Cleaning
import pandas as pd
df = pd.read_csv("dublin_properties_with_area_transport_distance1.csv")
print(f"Loaded {len(df)} listings for cleaning")
df['price'] = df['price'].astype(str).str.replace(r'[^\d]', '', regex=True).astype(float)
df['Floor Area'] = df['Floor Area'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce')
df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
print(df.head())

# ---- Feature Engineering ----
df['sale_year'] = df['sale_date'].dt.year
df['sale_month'] = df['sale_date'].dt.month_name()
df['Has_Transport'] = df['Nearby_Transport'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')
df['price_per_m2'] = (df['price'] / df['Floor Area']).round(2)

# ---- Export Cleaned Data ----
df.to_csv("dublin_cleaned_finals.csv", index=False)
print("✅ Cleaned dataset ready for Power BI")


Loaded 18250 listings for cleaning
                                             address  county          area  \
0      10 Moorings Cove, Balbriggan Street, Skerries  Dublin      Skerries   
1      10 Churchfields Avenue, Mulhuddart, Dublin 15  Dublin    Mulhuddart   
2  12 Glencairn Drive, The Gallops, Leopardstown ...  Dublin  Leopardstown   
3          13 The Drive, Carberry Meadows, Balmoston  Dublin     Balmoston   
4           13 The Lawns, Kilmartin Groe, Hollystown  Dublin    Hollystown   

   sale_date house_type  sale_type  eircode  BER  Bedrooms  Bathrooms  \
0 2024-12-17  Apartment     Resale  K34P650   C3       2.0        1.0   
1 2024-12-17        NaN  New Build      NaN  NaN       NaN        NaN   
2 2024-12-17     Semi-D     Resale  D18E6C8   C1       4.0        3.0   
3 2024-12-17        NaN  New Build      NaN  NaN       NaN        NaN   
4 2024-12-17        NaN  New Build      NaN  NaN       NaN        NaN   

   Floor Area      price      Nearby_Transport  Distance_

In [2]:
import pandas as pd
df = pd.read_csv("dublin_cleaned_final.csv")
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33200 entries, 0 to 33199
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   address                      33200 non-null  object 
 1   county                       33200 non-null  object 
 2   area                         26353 non-null  object 
 3   sale_date                    33200 non-null  object 
 4   house_type                   18424 non-null  object 
 5   sale_type                    33200 non-null  object 
 6   eircode                      27091 non-null  object 
 7   BER                          18949 non-null  object 
 8   Bedrooms                     19656 non-null  float64
 9   Bathrooms                    19655 non-null  float64
 10  Floor Area                   10960 non-null  float64
 11  price                        33200 non-null  int64  
 12  Nearby_Transport             8323 non-null   object 
 13  Distance_from_Ci